In [1]:
#problem 2
import numpy as np
import pandas as pd
from scipy.stats import norm, t
from scipy.integrate import quad

# Fit a normal distribution with exponentially weighted variance
def fit_normal_ew(data, lambda_=0.97):
    ew_mean = data.ewm(alpha=1 - lambda_).mean().iloc[-1]
    ew_variance = data.ewm(alpha=1 - lambda_).var().iloc[-1]
    ew_std = np.sqrt(ew_variance)
    return ew_mean, ew_std

# VaR using a normal distribution with an exponentially weighted variance
def var_normal_ew(data, alpha=0.05, lambda_=0.97):
    mu, sigma = fit_normal_ew(data, lambda_)
    VaR = -norm.ppf(alpha, mu, sigma)
    return VaR

# ES using a normal distribution with an exponentially weighted variance
def es_normal_ew(data, alpha=0.05, lambda_=0.97):
    mu, sigma = fit_normal_ew(data, lambda_)
    VaR = var_normal_ew(data, alpha, lambda_)
    ES = -(mu - sigma * (norm.pdf(norm.ppf(alpha)) / alpha))
    return ES

# Fit a t distribution using MLE
def fit_general_t(data):
    nu, mu, sigma = t.fit(data)
    return mu, sigma, nu

# VaR using a MLE fitted T distribution
def var_t(data, alpha=0.05):
    mu, sigma, nu = fit_general_t(data)
    VaR = -t.ppf(alpha, nu, mu, sigma)
    return VaR

# ES using a MLE fitted T distribution
def es_t(data, alpha=0.05):
    mu, sigma, nu = fit_general_t(data)
    VaR = var_t(data, alpha)
    ES, _ = quad(lambda x: x * t.pdf(x, df=nu, loc=mu, scale=sigma), -np.inf, -VaR)
    ES /= -alpha
    return ES

# VaR using historical simulation
def var_historical(data, alpha=0.05):
    VaR = -np.percentile(data, alpha*100)
    return VaR

# ES using historical simulation
def es_historical(data, alpha=0.05):
    VaR = var_historical(data, alpha)
    ES = -data[data <= -VaR].mean()
    return ES

data = pd.read_csv('/Users/queenieliu/FinTech545_Spring2024/Week05/Project/problem1.csv')['x']

# Calculate VaR and ES using the three methods
alpha = 0.05
lambda_ = 0.97

VaR_normal_ew = var_normal_ew(data, alpha, lambda_)
ES_normal_ew = es_normal_ew(data, alpha, lambda_)

VaR_t_dist = var_t(data, alpha)
ES_t_dist = es_t(data, alpha)

VaR_historical = var_historical(data, alpha)
ES_historical = es_historical(data, alpha)

# Create a DataFrame to display the results
results = pd.DataFrame({
    'Method': ['Normal (EW)', 'MLE T-Dist', 'Historical'],
    'VaR': [VaR_normal_ew, VaR_t_dist, VaR_historical],
    'ES': [ES_normal_ew, ES_t_dist, ES_historical]
})

print(results)

        Method       VaR        ES
0  Normal (EW)  0.091805  0.114919
1   MLE T-Dist  0.076476  0.113218
2   Historical  0.075981  0.116777


In [9]:
#Problem 3
from scipy.stats import norm, t

def return_calculate(prices, method='ARS', dateColumn='Date'):
    # Exclude the date column from the calculations
    tickers = [col for col in prices.columns if col != dateColumn]
    prices = prices[tickers] # The dataframe is now with no date column
    # Classical Brownian Motion
    if method == 'CBM':
        prices = prices.diff().dropna()
    # Arithmetic Return System
    elif method == 'ARS':
        prices = (prices - prices.shift(1)) / prices.shift(1)
        prices = prices.dropna()
    # Geometric Brownian Motion
    elif method == 'GBM':
        prices = np.log(df).diff().dropna()
    else:
        raise ValueError(f"method: {method} must be in (\"CBM\",\"ARS\",\"GBM\")")
    
    return prices

def simulatePCA(N, prices, mean=None, seed=1234, pctExp=1):
    # Error Checking
    m, n = prices.shape
    if n != m:
        raise ValueError(f"Covariance Matrix is not square ({n},{m})")
    # Initialize output
    out = np.zeros((N, n))
    # Set mean
    if mean is None:
        mean = np.zeros(n)
    else:
        if len(mean) != n:
            raise ValueError(f"Mean ({len(mean)}) is not the size of cov ({n},{n})")
    
    eigenvalues, eigenvectors = np.linalg.eig(prices)
    
    # Get the indices that would sort eigenvalues in descending order
    indices = np.argsort(eigenvalues)[::-1]
    # Sort eigenvalues
    eigenvalues = eigenvalues[indices]
    # Sort eigenvectors according to the same order
    eigenvectors = eigenvectors[:, indices]
    
    tv = np.sum(eigenvalues)
    posv = np.where(eigenvalues >= 1e-8)[0]
    if pctExp <= 1:
        nval = 0
        pct = 0.0
        # How many factors needed
        for i in posv:
            pct += eigenvalues[i] / tv
            nval += 1
            if pct >= pctExp:
                break
    
     # If nval is less than the number of positive eigenvalues, truncate posv
    if nval < len(posv):
        posv = posv[:nval]
        
    # Filter eigenvalues based on posv
    eigenvalues = eigenvalues[posv]
    eigenvectors = eigenvectors[:, posv]
    
    B = eigenvectors @ np.diag(np.sqrt(eigenvalues))
    
    np.random.seed(seed)
    rand_normals = np.random.normal(0.0, 1.0, size=(N, len(posv)))
    out = np.dot(rand_normals, B.T) + mean
    
    return out.T

def simulate_copula(portfolio, returns):
    portfolio['CurrentValue'] = portfolio['Holding'] * portfolio['Starting Price']
    models = {}
    uniform = pd.DataFrame()
    standard_normal = pd.DataFrame()
    
    for stock in portfolio["Stock"]:
        # If the distribution for the model is normal, fit the data with normal distribution
        if portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0] == 'Normal':
            models[stock] = norm.fit(returns[stock])
            mu, sigma = norm.fit(returns[stock])
            # Transform the observation vector into a uniform vector using CDF
            uniform[stock] = norm.cdf(returns[stock], loc=mu, scale=sigma)
            # Transform the uniform vector into a Standard Normal vector usig the normal quantile function
            standard_normal[stock] = norm.ppf(uniform[stock])
            
        # If the distribution for the model is t, fit the data with normal t
        elif portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0] == 'T':
            models[stock] = t.fit(returns[stock])
            nu, mu, sigma = t.fit(returns[stock])
            # Transform the observation vector into a uniform vector using CDF
            uniform[stock] = t.cdf(returns[stock], df=nu, loc=mu, scale=sigma)
            # Transform the uniform vector into a Standard Normal vector usig the normal quantile function
            standard_normal[stock] = norm.ppf(uniform[stock])
        
    # Calculate Spearman's correlation matrix
    spearman_corr_matrix = standard_normal.corr(method='spearman')
    
    simulate_time = 10000
    
    # Use the PCA to simulate the multivariate normal
    simulations = simulatePCA(simulate_time, spearman_corr_matrix)
    simulations = pd.DataFrame(simulations.T, columns=[stock for stock in portfolio["Stock"]])
    # Transform the simulations into uniform variables using standard normal CDF
    uni = norm.cdf(simulations)
    uni = pd.DataFrame(uni, columns=[stock for stock in portfolio["Stock"]])
    simulatedReturns = pd.DataFrame()
    # Transform the uniform variables into the desired data using quantile
    for stock in portfolio["Stock"]:
        # If the distribution for the model is normal/t, use the quantile of the normal/t distribution
        if portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0] == 'Normal':
            mu, sigma = models[stock]
            simulatedReturns[stock] = norm.ppf(uni[stock], loc=mu, scale=sigma)
        elif portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0] == 'T':
            nu, mu, sigma = models[stock]
            simulatedReturns[stock] = t.ppf(uni[stock], df=nu, loc=mu, scale=sigma)
    
    simulatedValue = pd.DataFrame()
    pnl = pd.DataFrame()
    # Calculate the daily prices for each stock
    for stock in portfolio["Stock"]:
        currentValue = portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0]
        simulatedValue[stock] = currentValue * (1 + simulatedReturns[stock])
        pnl[stock] = simulatedValue[stock] - currentValue
        
    risk = pd.DataFrame(columns = ["Stock", "VaR95", "ES95", "VaR95_Pct", "ES95_Pct"])
    w = pd.DataFrame()

    for stock in pnl.columns:
        i = risk.shape[0]
        risk.loc[i, "Stock"] = stock
        risk.loc[i, "VaR95"] = -np.percentile(pnl[stock], 5)
        risk.loc[i, "VaR95_Pct"] = risk.loc[i, "VaR95"] / portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0]
        risk.loc[i, "ES95"] = -pnl[stock][pnl[stock] <= -risk.loc[i, "VaR95"]].mean()
        risk.loc[i, "ES95_Pct"] = risk.loc[i, "ES95"] / portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0]
        
        # Determine the weights for the two stock
        w.at['Weight', stock] = portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0] / portfolio['CurrentValue'].sum()
        
    # Calculate the total pnl
    pnl['Total'] = 0
    for stock in portfolio["Stock"]:
        pnl['Total'] += pnl[stock]
    
    i = risk.shape[0]
    risk.loc[i, "Stock"] = 'Total'
    risk.loc[i, "VaR95"] = -np.percentile(pnl['Total'], 5)
    risk.loc[i, "VaR95_Pct"] = risk.loc[i, "VaR95"] / portfolio['CurrentValue'].sum()
    risk.loc[i, "ES95"] = -pnl['Total'][pnl['Total'] <= -risk.loc[i, "VaR95"]].mean()
    risk.loc[i, "ES95_Pct"] = risk.loc[i, "ES95"] / portfolio['CurrentValue'].sum()

    return risk
#Calculation for outcomes
prices = pd.read_csv('/Users/queenieliu/FinTech545_Spring2024/Week05/Project/DailyPrices.csv')
returns = return_calculate(prices)
returns = returns - returns.mean()
portfolio = pd.read_csv('/Users/queenieliu/FinTech545_Spring2024/Week05/Project/portfolio.csv')
portfolio.loc[portfolio['Portfolio'].isin(['A', 'B']), 'Distribution'] = 'T'
portfolio.loc[portfolio['Portfolio'] == 'C', 'Distribution'] = 'Normal'
for stock in portfolio["Stock"]:
    portfolio.loc[portfolio['Stock'] == stock, 'Starting Price'] = prices.iloc[-1][stock]

simulate_copula(portfolio, returns)

,Stock,VaR95,ES95,VaR95_Pct,ES95_Pct
0,AAPL,317.514095,414.053098,0.036341,0.04739
1,TSLA,143.216143,182.481725,0.06908,0.088019
2,JPM,267.057203,354.041985,0.029716,0.039395
3,HD,265.106917,370.74386,0.031116,0.043515
4,BAC,241.428817,337.469293,0.032339,0.045204
...,...,...,...,...,...
95,LRCX,396.750417,494.57176,0.054972,0.068526
96,MO,236.130993,298.985396,0.025872,0.032758
97,LMT,340.204083,431.648095,0.026804,0.034008
98,TFC,237.339103,301.881741,0.033233,0.042271


In [13]:
portfolio_a = portfolio.loc[portfolio["Portfolio"] == "A"].copy()
risk_a = simulate_copula(portfolio_a, returns)
print("The risk matrix for Portfolio A is:\n", risk_a)
print("\n")

portfolio_b = portfolio.loc[portfolio["Portfolio"] == "B"].copy()
risk_b = simulate_copula(portfolio_b, returns)
print("The risk matrix for Portfolio B is:\n", risk_b)
print("\n")

portfolio_c = portfolio.loc[portfolio["Portfolio"] == "C"].copy()
risk_c = simulate_copula(portfolio_c, returns)
print("The risk matrix for Portfolio C is:\n", risk_c)
print("\n")

The risk matrix for Portfolio A is:
     Stock        VaR95         ES95 VaR95_Pct  ES95_Pct
0    AAPL   323.111393   418.913228  0.036981  0.047946
1    TSLA   141.633416   180.066431  0.068316  0.086854
2     JPM   271.588181   357.070898  0.030221  0.039732
3      HD   262.209692   368.862159  0.030776  0.043294
4     BAC   250.582991   344.111929  0.033566  0.046094
5     XOM   570.372501   742.656859  0.035628   0.04639
6    AVGO   389.639611    500.02063  0.038238   0.04907
7     PEP   190.238318   278.956372  0.019317  0.028325
8     TMO   320.022494   439.492836  0.033088  0.045441
9   CMCSA   222.564299   311.217299   0.03027  0.042328
10   META   334.293002   498.559162  0.056936  0.084914
11    ACN   278.582841   363.146999  0.033687  0.043913
12   INTC   200.484168   278.384783  0.040166  0.055773
13   PYPL   248.158889   333.421788  0.055517  0.074592
14    MRK   269.273891   386.853121  0.020514  0.029471
15      T   173.911404   265.308044  0.024912  0.038005
16    LOW  